In [2]:
import os
import numpy as np
import re
import pandas as pd
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import pickle
from torch.nn.utils.rnn import pad_sequence
from unicodedata import normalize
from keras.preprocessing.text import Tokenizer
from torchvision import transforms, datasets
from tqdm import tqdm
from time import sleep
import matplotlib.pyplot as plt

ImportError: cannot import name 'PILLOW_VERSION' from 'PIL' (/home/speed/joao.pedrosa/miniconda3/lib/python3.7/site-packages/PIL/__init__.py)

In [2]:
# Global variables
REBUILD_DATA = True
TOP_WORDS = 10000
EPOCHS = 30
BATCH_SIZE = 256
#Essa variável aqui é o número de posições que os vetores tem ao usar o pad_sequences. Não uso ela pra nada ainda mas talvez eu venha a usar, por isso ela está aqui.
# SEQ_SIZE = 390

In [1]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on a GPU :D")
else:
    device = torch.device("cpu")
    print("Running on a CPU :/")

NameError: name 'torch' is not defined

In [12]:
# Oi, Derick
# To comentando como diálogo porque acho mais fácil explicar o código assim. Se o comentário estiver errado é pq essa é a parte que eu entendi errado.
class Net(nn.Module):
    def __init__(self, seq_size):
        
        print("Building NN...")
        embedding_dim = 128
        lstm_out_dim = 128
        num_embeddings = 7500
        num_of_classes = 74
        
        super().__init__()
        #Camada de Embedding, o padding_idx é um argumento que eu descobri que é usada para falar para a camada que os números no fim de cada vetor são apenas lixo
        self.l1 = nn.Embedding(num_embeddings, embedding_dim, padding_idx = 0)
        #Eu não entendo muito bem o que essa camada faz. Pelo que eu entendi é algo probabilístico. Mas ela n altera o shape.
        self.l2 = nn.Dropout(p=0.4)
        #A LSTM recebe os Embeddings e cospe o mesmo número de vetores que eu passei para ela. Não sei se eu deveria alterar o número de camadas da LSTM.
        #Se usar menos de 2 não dá pra colocar Dropout pq o Dropout é aplicado em todas as camadas menos na última.
        self.l3 = nn.LSTM(embedding_dim, lstm_out_dim, dropout = 0.2, num_layers = 2)
        #É o seguinte. Como as dimensões de entrada são estáticas, eu adicionei elas manualmente na camada linear para conseguir fazer o flatten.
        self.l4 = nn.Flatten()
        #Dimensao do vetor de entrada X dimensao da lstm
        self.l5 = nn.Linear(seq_size * lstm_out_dim, num_of_classes)
        
    
    def forward(self, x):
        #Aqui eu só to passando o input pelas camadas mesmo
        x    = self.l1(x)
        x    = self.l2(x)
        #A camada de LSTM retorna uma tupla, o vetor que eu quero é a primeira posição da tupla, por isso recebo assim.
        #Acho que a segunda camada da LSTM só é util ao passar de uma camada da LSTM para a outra.
        x, _ = self.l3(x)
        x    = self.l4(x)
        x    = self.l5(x)
        #Aqui eu aplico o softmax. Especifico o número de dimensões para ser um e tal. Não sei o que não está funcionando :c.
        x    = F.softmax(x, dim = 1)
            
        return x        

In [13]:
def clean_text(x):
    if type(x) is str:
        pattern = r'[^a-zA-z0-9!:.,?\s]'
        x = normalize('NFKD', x).encode('ASCII', 'ignore').decode('ASCII')
        x = re.sub(pattern, '', x)
        return x
    else:
        return ""

In [60]:
def load_data(data = "train"):
    print("Loading data:\n")
    print("Ids...")
    ids_achados    = pd.read_csv('../../../data/greenZoneIds.csv', index_col = 0)
    print("Text...")
    dados_texto    = pd.read_csv('../../../data/DATA_LAUDOS_TEXTO_formato1', sep = ";")
    print("Labels...")
    resultLabels   = pd.read_csv('../../../data/resultados/resultLabels.csv')
    print("Data loaded!\n")
    
    print("Preprocessing data:\n")
    print("Selecting slices...")
    if data == "test":
        dados_texto    = dados_texto[~dados_texto['ID_EXAME'].isin(ids_achados['id'])]
    else:
        dados_texto    = dados_texto[dados_texto['ID_EXAME'].isin(ids_achados['id'])]
        resultLabels   = resultLabels[resultLabels['idExame'].isin(ids_achados['id'])]
    # dados_validacao = dados_texto[dados_texto['ID_EXAME'].isin(ids_achados['id'][-400000:])]

    text  = dados_texto['CONTEUDO']
    labels = [row[1:-1] for row in resultLabels['label']]
    labels = [label.split(',') for label in labels]
    labels = [[1 if word == " True" or word == "True" else 0 for word in row] for row in labels]
    text   = text[:-50000]
    labels = labels[:-50000]
    
    return text, labels

In [59]:
def tokenize(text, tokenizer, fit = False):
    print("Tokenizing...")
    # Creating vocabulary
    if fit:
        tokenizer.fit_on_texts(text)
    # Vectorizing text
    train_X   = tokenizer.texts_to_sequences(text)
    # Saving tokenizer
    with open('../../../data/trained_models/pytorch_tokenizer.pickle', 'wb') as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return train_X

In [58]:
if REBUILD_DATA:
        text, labels = load_data()
        text         = [clean_text(i) for i in text]
        tokenizer = Tokenizer(num_words = TOP_WORDS, split = ' ')
        train_X   = tokenize(text, tokenizer, fit = True)
        np.save("../../../data/training_data/training_data_X.npy", train_X)
else:
        train_X = np.load('../../../data/training_data/training_data_X.npy', allow_pickle = True)

Loading data...
Ids...
Text...
Terraforming data...
1/2...
2/2...


In [62]:
train_y  = labels
print("Transforming lists into tensors...")
train_X  = [torch.Tensor(i).type(torch.LongTensor) for i in train_X]
train_X  = pad_sequence(train_X, batch_first=True).type(torch.LongTensor)
train_y  = torch.Tensor(train_y)
seq_size = len(train_X[0])
print("Data preprocessed!\n")
print("Number of input dimensions: ", seq_size)

Transforming lists into tensors...
Data preprocessed!

Number of input dimensions:  422


In [64]:
net = Net(seq_size).to(device)
optimizer = optim.Adam(net.parameters(), lr = 0.0005)
loss_function = nn.BCELoss()
# Training the model
print("\nAlexa, play eye of the tiger. It's train time!\n\n")
for epoch in range(EPOCHS):
    print("Epoch ", epoch)
    for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
#         print(i, i+BATCH_SIZE)
        batch_X = train_X[i:i+BATCH_SIZE]
        batch_y = train_y.squeeze()[i:i+BATCH_SIZE]

        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        net.zero_grad()
        outputs = net(batch_X)
        loss    = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()

    print("End of Epoch ", epoch, "!\n Loss: ", loss,"\n")

print("Finished training! Loss: ", loss)
print("\nSaving model...")
torch.save(net.state_dict(), "../../../data/trained_models/pytorch_checkpoint_2.pth")
print("Finish!")


Alexa, play eye of the tiger. It's train time!


Building NN...


  0%|          | 0/2446 [00:00<?, ?it/s]

Epoch  0


  3%|▎         | 77/2446 [00:06<03:16, 12.08it/s]


KeyboardInterrupt: 